Ce script permet d’évaluer les performances des tokeniseurs (SpaCy, Stanza et BERT) sur le corpus bruité.
Il compare les séquences de tokens générées par chaque tokeniseur avec la séquence de référence, en utilisant la métrique Exact Match (EM), après il calcule la précison , rappel et F mesure .
À noter : dans notre configuration, BERT ne segmente pas les phrases en mots mais conserve chaque entrée comme un seul token. Cela a un impact sur l’évaluation, puisque ses résultats ne sont pas directement comparables à ceux de SpaCy et Stanza

In [ ]:
import json
import csv
from collections import defaultdict

def evaluate_tokenization(reference_tokens, predicted_tokens):
    """Compare deux listes de tokens et calcule Exact Match, Précision, Rappel, F1"""
    exact_match = 1 if reference_tokens == predicted_tokens else 0

    # Convert lists to sets for efficient comparison
    ref_set = set(reference_tokens)
    pred_set = set(predicted_tokens)

    true_positives = len(ref_set.intersection(pred_set))
    precision = true_positives / len(pred_set) if pred_set else 0
    recall = true_positives / len(ref_set) if ref_set else 0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0

    return exact_match, precision, recall, f1


# Charger ton fichier JSON
with open("dataset_bruit_tokenized.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Fichier CSV détaillé
with open("dataset_bruit_tokenizes.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Phrase", "Bruit", "Tokenizer", "ExactMatch", "Précision", "Rappel", "F1"])

    # Pour les moyennes
    scores = defaultdict(list)

    # Define the noise levels to iterate over
    noise_levels = ["original", "bruit_faible", "bruit_moyen", "bruit_fort"]
    tokenizers = ["spacy", "stanza", "bert"]

    for item in data:
        reference = item["reference_tokens"]

        for bruit_level in noise_levels:
            phrase = item[bruit_level] 

            for tokenizer_name in tokenizers:
                # Construct the key for the predicted tokens
                predicted_key = tokenizer_name
                if bruit_level != "original":
                    predicted_key = f"{tokenizer_name}" # Predicted tokens are grouped by tokenizer name in the JSON

                predicted = item[predicted_key][bruit_level] # Access the tokens for the specific noise level

                exact, prec, rec, f1 = evaluate_tokenization(reference, predicted)

                writer.writerow([phrase, bruit_level, tokenizer_name, exact, prec, rec, f1])

                # Sauvegarde pour moyennes
                scores[(bruit_level, tokenizer_name)].append((exact, prec, rec, f1))


# Calcul des moyennes
with open("resume_tokenizers.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Bruit", "Tokenizer", "ExactMatch_moy", "Précision_moy", "Rappel_moy", "F1_moy"])

    for (bruit, tokenizer), vals in scores.items():
        if vals:
            exacts, precs, recs, f1s = zip(*vals)
            writer.writerow([
                bruit, tokenizer,
                sum(exacts)/len(exacts),
                sum(precs)/len(precs),
                sum(recs)/len(recs),
                sum(f1s)/len(f1s)
            ])

print(" Résultats détaillés : dataset_bruit_tokenizes.csv")
print(" Résumé global : resume_tokenizers.csv")

✅ Résultats détaillés : dataset_bruit_tokenizes.csv
✅ Résumé global : resume_tokenizers.csv
